In [1]:
import datajoint as dj
import numpy as np
import time
import pymeshfix
import os
import trimesh
from pathlib import Path

import cgal_Segmentation_Module as csm

#for supressing the output
import contextlib

#for fixing the space issue with the CGAL:readoff
import csv

from collections import Counter

import networkx as nx

ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

Connecting celiib@10.28.0.34:3306


In [ ]:
# def segment_Whole_Neuron(clusters=3,smoothness=0.20,filename="None",key=dict()):
#     original_start_time = time.time() 
#     start_time = time.time()
#     if filename != "None":
#         if not os.path.isfile(filename):
#             print("neuron filename not exist")
#             return "failure"


#         my_mesh = trimesh.load_mesh(filename)
#         verts = my_mesh.vertices
#         faces = my_mesh.faces

#     elif "segment_id" in key.keys():
        
        
#         segment_id = key["segment_id"]
#         decimation_ratio = key.pop("decimation_ratio",0.35)
#         segmentation = key.pop("segmentation",2)
        
#         primary_key = dict(segmentation=segmentation,decimation_ratio=decimation_ratio,segment_id=neuron_ID)
#         neuron_data = (ta3p100.CleansedMesh & primary_key).fetch1()

#         print(neuron_data)
#         verts = neuron_data['vertices'].astype(dtype=np.int32)
#         faces = neuron_data['triangles'].astype(dtype=np.uint32)
#         return verts,faces
        


#     else:
#         print("No valid key or filename given")
#         return "failure"
    
    
#     #run pymeshfix on the mesh
#     #meshfix = pymeshfix.MeshFix(verts,faces)
    
#     #write the output file to an off file
# segment_id=648518346349471500
# key = dict(segment_id=segment_id)
# verts,faces = segment_Whole_Neuron(key=key)



In [2]:
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    
    file_loc = pathlib.Path.cwd() / "temp"
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [ ]:
ta3p100.CleansedMesh() & "n_vertices<50000"

In [3]:
key = dict(segment_id=648518346349472574)
#key = dict(segment_id=648518346349472574)
print("Loading mesh from datajoint- id: " + str(key["segment_id"]))
segment_id = key["segment_id"]
decimation_ratio = key.pop("decimation_ratio",0.35)
segmentation = key.pop("segmentation",2)

primary_key = dict(segmentation=segmentation,decimation_ratio=decimation_ratio,segment_id=segment_id)
neuron_data = (ta3p100.CleansedMesh & primary_key).fetch1()

print(neuron_data)
vertices = neuron_data['vertices']#.astype(dtype=np.float)
faces = neuron_data['triangles']#.astype(dtype=np.int)
write_Whole_Neuron_Off_file(segment_id,vertices,faces)


Loading mesh from datajoint- id: 648518346349472574
OrderedDict([('segmentation', 2), ('segment_id', 648518346349472574), ('decimation_ratio', Decimal('0.35')), ('n_vertices', 560132), ('n_triangles', 1134283), ('vertices', array([[308382.71875   , 153810.40625   ,  40942.4765625 ],
       [308429.34375   , 153877.75      ,  40947.5234375 ],
       [308451.25      , 153913.5625    ,  40888.29296875],
       ...,
       [475551.8125    , 236005.6875    ,  82891.734375  ],
       [475571.625     , 236233.859375  ,  82868.5390625 ],
       [475601.75      , 236416.75      ,  82883.3828125 ]])), ('triangles', array([[237021, 237023, 236526],
       [238728, 239014, 238941],
       [238728, 238941, 238474],
       ...,
       [511531, 510694, 510702],
       [542792, 542786, 542791],
       [542034, 542786, 542792]], dtype=uint32))])
Done writing OFF file


('/notebooks/19_Whole_Neuron_Seg_Revisited/temp/neuron_648518346349472574',
 'neuron_648518346349472574',
 '/notebooks/19_Whole_Neuron_Seg_Revisited/temp')

In [4]:
trimesh.load_mesh("temp/neuron_648518346349472574.off")

In [ ]:
meshfix2 = pymeshfix.MeshFix(vertices,faces)
meshfix2.repair(verbose=False,joincomp=True,remove_smallest_components=False)


In [ ]:
write_Whole_Neuron_Off_file(str(segment_id) + "_pymesh_fixed",meshfix2.v,meshfix2.f)

In [ ]:
meshfix = pymeshfix.MeshFix(vertices,faces)
meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")

In [ ]:
write_Whole_Neuron_Off_file(str(648518346349473044) + "_fixed",meshfix.v,meshfix.f)

In [ ]:
meshfix.write("neuron_648518346349473044_fixed.off")

In [ ]:
classifier.faces[np.where(classifier.labels_list == 13)]

In [ ]:
classifier.min_max

In [ ]:
axis = 1
neighbors_list = [13]
min_max = {ni:[300000,-300000] for ni in neighbors_list}

for i,ll in enumerate(classifier.labels_list):
    if ll in neighbors_list:
        verts_from_faces = classifier.faces[i]
        for vert in verts_from_faces:
            real_vert = classifier.vertices[vert]
            if real_vert[axis] < min_max[ll][0]:
                min_max[ll][0] = real_vert[axis]
            if real_vert[axis] > min_max[ll][1]:
                min_max[ll][1] = real_vert[axis]
print(min_max)


In [ ]:
#classifier.mesh.submesh(classifier.faces[np.where(classifier.labels_list == 13)])
#classifier.mesh.submesh(np.where(classifier.labels_list == 13))[0].show()
np.min(classifier.vertices[classifier.faces[np.where(classifier.labels_list == 13)].ravel()][:,1])

In [ ]:
np.max(self.faces[np.where(self.labels_list == label)],axis=0)[1]

In [ ]:
#class that will handle the whole neuron segmentation:
class WholeNeuronClassifier(object):
    
    
    def generate_verts_to_face_dictionary(self,labels_list=[]):
        if len(labels_list) <= 1:
            print("len(labels_list) <= 1")
            labels_list = self.labels_list
        
        verts_to_Face = {i:[] for i,vertex in enumerate(self.vertices)}
        verts_to_Label = {i:[] for i,vertex in enumerate(self.vertices)}


        for i,verts in enumerate(self.faces):
            #get the vertices

            for vertex in verts:
                verts_to_Face[vertex].append(i)

        #use the verts to face to create the verts to label dictionary
        for vert,face_list in verts_to_Face.items():
            diff_labels = [labels_list[fc] for fc in face_list]
            #print(list(set(diff_labels)))
            verts_to_Label[vert] = list(set(diff_labels))
            
        self.verts_to_Face = verts_to_Face
        self.verts_to_Label = verts_to_Label
        
        print("inside generate verts_to_face")

        return
    
#     def generate_verts_to_face_dictionary(self):
#         """
#         generates the mapping of vertices to the faces that are touching it
        
#         """
#         verts_to_Face = {}

#         #initialize the lookup dictionary as empty lists
#         faces_raw = self.mesh.faces
#         verts_raw = self.mesh.vertices
        
#         for i,pre_vertex in enumerate(verts_raw):
#             verts_to_Face[i] = []
        

#         for i,verts in enumerate(faces_raw):
#             #add the index to the list for each of the vertices
#             for vertex in verts:
#                 verts_to_Face[vertex].append(i)

#         return verts_to_Face
    
    def __init__(self,mesh_file_location="",file_name="",key=dict()):
    #import the mesh

        full_path = str(Path(mesh_file_location) / Path(file_name))
        self.mesh = trimesh.load_mesh(full_path)
        
        #get the vertices to faces lookup table
        
        original_start_time = time.time() 
        start_time = time.time()
        if os.path.isfile(full_path):
            print("Loading mesh from " + str(full_path))
    
            my_mesh = trimesh.load_mesh(full_path)
            vertices = my_mesh.vertices
            faces = my_mesh.faces

        elif "segment_id" in key.keys():

            print("Loading mesh from datajoint- id: " + str(key["segment_id"]))
            segment_id = key["segment_id"]
            decimation_ratio = key.pop("decimation_ratio",0.35)
            segmentation = key.pop("segmentation",2)

            primary_key = dict(segmentation=segmentation,decimation_ratio=decimation_ratio,segment_id=segment_id)
            neuron_data = (ta3p100.CleansedMesh & primary_key).fetch1()

            print(neuron_data)
            vertices = neuron_data['vertices']#.astype(dtype=np.int32)
            faces = neuron_data['triangles']#.astype(dtype=np.uint32)
            


        else:
            print("No valid key or filename given")
            return "failure"
        
        self.vertices = vertices
        self.faces = faces
        #if need to generate verts to face
        #self.verts_to_Face = self.generate_verts_to_face_dictionary()
        
        """Ignoring for now so don't have to wait for pymeshfix to run"""
#         start_time = time.time()
#         print("Starting pymeshfix to clean Mesh")
#         meshfix = pymeshfix.MeshFix(vertices,faces)
#         meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
#         print(f"Step 2: Pymesh shrinkwrapping: {time.time() - start_time}")
        
#         self.verts = meshfix.v
#         self.faces = meshfix.f

        trimesh_object = trimesh.Trimesh()
        trimesh_object.faces = self.faces
        trimesh_object.vertices = self.vertices
        self.mesh = trimesh_object
        self.mesh_file_location = mesh_file_location
        self.file_name = file_name
        print(f"Loading mesh and Pymesh fix: \
        {np.round(time.time() - start_time,5)} seconds")
    
    #write the faces and vertices to an off file
    def export_self_mesh(self,file_path_and_name):
        with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
            self.mesh.export(file_path_and_name)
    
        
    def load_cgal_segmentation(self,clusters=3,smoothness=0.20):
        """
        Runs the cgal surface mesh segmentation on the mesh object
        
        """
        
        #have to write the new mesh to an off file
        new_mesh_file_path_and_name = str(Path(self.mesh_file_location) /
                                            Path(self.file_name[:-4] + "_fixed.off"))
        
        self.export_self_mesh(new_mesh_file_path_and_name)
        #add an extra end line to the off file
        with open(new_mesh_file_path_and_name,'a') as fd:
            fd.write("\n")
        
#         write_Whole_Neuron_Off_file(self.file_name[8:-4],self.mesh.vertices,
#                                                            self.mesh.faces)
        
    
        """skip the segmentation for now"""
        start_time = time.time()
#         print("\nStarting CGAL segmentation")
#         result = csm.cgal_segmentation(new_mesh_file_path_and_name[:-4],clusters,smoothness)
#         print(result)
        
        self.clusters = clusters
        self.smoothness = smoothness
        self.labels_file = str(Path(self.mesh_file_location) / Path(self.file_name[:-4] + "_fixed" + "-cgal_" + str(np.round(clusters,2)) + "_" + "{:.2f}".format(smoothness) + ".csv" ))  
        self.sdf_file = str(Path(self.mesh_file_location) / Path(self.file_name[:-4] + "_fixed" + "-cgal_" + str(clusters) + "_" + "{:.2f}".format(smoothness) + "_sdf.csv" ))  
        print(f"Step 2: CGAL segmentation total time ---- {np.round(time.time() - start_time,5)} seconds")

        
        
    #used for when not pulling from datajoint
    def get_cgal_data_and_label_local_optomized(self):
        
        labels_file = self.labels_file
        sdf_file = self.sdf_file
        
        #reads int the cgal labels for all of the faces
        triangles_labels = np.zeros(len(self.mesh.faces)).astype("int64")
        with open(labels_file) as csvfile:
            #print("inside labels file")

            for i,row in enumerate(csv.reader(csvfile)):
                triangles_labels[i] = int(row[0])


        #converts the cgal labels into a list that
        # starts at 0
        # progresses in order for all unique labels (so no numbers are skipped and don't have corresponding face)
        verts_raw = self.mesh.vertices
        faces_raw = self.mesh.faces
        #gets a list of the unique labels
        unique_segments = list(Counter(triangles_labels).keys())
        segmentation_length = len(unique_segments) 
        unique_index_dict = {unique_segments[x]:x for x in range(0,segmentation_length )}
        
        labels_list = np.zeros(len(triangles_labels)).astype("int64")
        for i,tri in enumerate(triangles_labels):

            #assembles the label list that represents all of the faces
            labels_list[i] = int(unique_index_dict[tri])

        #write thses new labels to a file
        with open(labels_file[:-4] + "_revised.csv",mode="w") as csvfile:
            csv_writer = csv.writer(csvfile,delimiter=',')
            for i in labels_list:
                csv_writer.writerow([i])

        #print("done with cgal_segmentation")

        #----------------------now return a dictionary of the sdf values like in the older function get_sdf_dictionary
        #get the sdf values and store in sdf_labels
        sdf_labels = np.zeros(len(labels_list)).astype("float")
        with open(sdf_file) as csvfile:

            for i,row in enumerate(csv.reader(csvfile)):
                sdf_labels[i] = float(row[0])

        
        sdf_temp_dict = {}
        for i in range(0,segmentation_length):
            sdf_temp_dict[i] = []
        
        #print("sdf_temp_dict = " + str(sdf_temp_dict))
        #print("sdf_labels = " + str(sdf_labels))
        #iterate through the labels_list
        for i,label in enumerate(labels_list):
            sdf_temp_dict[label].append(sdf_labels[i])
        #print(sdf_temp_dict)

        #now calculate the stats on the sdf values for each label
        sdf_final_dict = {}
        
        for dict_key,value in sdf_temp_dict.items():

            sdf_final_dict[dict_key] = dict(median=np.median(value),mean=np.mean(value),max=np.amax(value))


        self.sdf_final_dict = sdf_final_dict
        self.labels_list = labels_list
        self.labels_list_counter = Counter(labels_list)
    
        adjacency_labels = self.labels_list[self.mesh.face_adjacency]
        
        self.adjacency_labels_col1, self.adjacency_labels_col2 = adjacency_labels.T
        #generate the vertices labels
        self.generate_verts_to_face_dictionary(labels_list)
        
        return 
        
    
    
    def get_highest_sdf_part(self,size_threshold=3000,exclude_label=None):
        """
        Based ont the sdf data and the labels data,
        Finds the label with the highest median,
            label with highest max,
            label with highest mean sdf value
        
        *** but only for those that meet the certain threshold ***
        
        
        """
        high_median_val = 0
        high_median = -1
        high_mean_val = 0
        high_mean = -1
        high_max_val = 0
        high_max = -1



        #gets all of the labels
        my_list = Counter(self.labels_list)
        my_list_keys = list(my_list.keys())
        if exclude_label != None:
            my_list_keys.remove(exclude_label)

        #OPTOMIZE
        
        for x in my_list_keys:
            #print("x = " + str(x))
            #print("high_median_val = " + str(high_median_val))
            #print('sdf_final_dict[x]["median"] = ' + str(sdf_final_dict[x]["median"]))
            if self.sdf_final_dict[x]["median"] > high_median_val and my_list[x] > size_threshold:
                high_median = x
                high_median_val = self.sdf_final_dict[x]["median"]
            if self.sdf_final_dict[x]["mean"] > high_mean_val  and my_list[x] > size_threshold:
                high_mean = x
                high_mean_val = self.sdf_final_dict[x]["mean"]
            if self.sdf_final_dict[x]["max"] > high_max_val  and my_list[x] > size_threshold:
                high_max = x
                high_max_val = self.sdf_final_dict[x]["max"]


        self.highest_vals= [high_median,high_median_val,high_mean,high_mean_val,high_max,high_max_val]
        
        self.high_median = self.highest_vals[0]
        return 
    

    #getting bad connection data from here
    def get_graph_structure(self):
        """
        For each unique label gets:
        1) all neighbors
        2) number of faces belonging to that label

        """

        connections = {label_name:[] for label_name in self.labels_list_counter.keys()}
        mesh_Number = {label_name:number for label_name,number in self.labels_list_counter.items()}
        #label_vert_stats = {label_name:[300000,-300000] for label_name in Counter(labels_list).keys()}

        #verts to label curently is the has every vertex and the labels it is toughing in a list
        for verts,total_labels in self.verts_to_Label.items():
            if len(total_labels) > 1: #if more than one label
                for face in total_labels:
                    for fc in [v for v in total_labels if v != face]:
                        if fc not in connections[face]:
                            connections[face].append(fc)

        self.connections = connections
        self.mesh_Number = mesh_Number
        
        print("INSIDE GET GRAPH STRUCTURE and connections[5] = " + str(connections[5]))

        return 
    
#     def find_max_min_z_vals(self,neighbors_list,axis=1):
#         """
#         for each of the neighbors of the soma and including the soma,
#         will find the maximum and minimum value for the axis direction
#         """
#         min_max = {ni:[300000,-300000] for ni in neighbors_list}

#         for i,ll in enumerate(self.labels_list):
#             if ll in neighbors_list:
#                 verts_from_faces = self.faces[i]
#                 for vert in verts_from_faces:
#                     real_vert = self.vertices[vert]
#                     if real_vert[axis] < min_max[ll][0]:
#                         min_max[ll][0] = real_vert[axis]
#                     if real_vert[axis] > min_max[ll][1]:
#                         min_max[ll][1] = real_vert[axis]
#         self.min_max = min_max
#         return min_max
    
    
   
    def find_Soma_Caps(self,soma_index,min_width=0.23,max_faces=6000,max_n_connection=6,large_sized_convex=3):
        #get the soma neighbors
        soma_neighbors = self.connections[soma_index]
        
        total_soma_caps = []
        for i in soma_neighbors:
            soma_cap = True
            
            #collect the mesh of the cap
            submesh = self.mesh.submesh(np.where(self.labels_list == i))[0]

            mean_convex = abs(np.mean(trimesh.convex.adjacency_projections(submesh)))
            n_faces = len(submesh.faces)
            width_data = self.sdf_final_dict[i]
            width_data_median = self.sdf_final_dict[i]["median"]
            n_connections = len(self.connections[i])
            
#             131 mean_convex = 7.214328769367434, n_faces=483, width_data=0.256008, n_connections=2
#             129 mean_convex = 9.830215623946183, n_faces=430, width_data=0.2710475, n_connections=2
            
#             if i in [129,131]:
#                 print(f" {i} mean_convex = {mean_convex}, n_faces={n_faces}, width_data={width_data_median}, n_connections={n_connections}")
                
                
            
            if width_data["median"] < min_width or n_faces>max_faces or n_connections>max_n_connection: 
                soma_cap = False
            
            #use the convex data if size is really big:
            if n_faces > 1500:
                if mean_convex > 5:
                    #print(f" {i} Doesn't meet second pass")
                    soma_cap = False
            
            if soma_cap == True:
                total_soma_caps.append(i)
            
#             print(total_soma_caps)
        
#         print(total_soma_caps)
        #for all the soma caps replace the labels list with soma_index and recompute neighbors and connections:
        if len(total_soma_caps) > 0:
            print(f"found {len(total_soma_caps)} soma caps and replacing labels")
            start_time = time.time()
            self.labels_list[np.where(np.isin(self.labels_list,total_soma_caps))] = soma_index
            
            #call the functions to recompute the connections/neighbors and others (but don't need to generate SDF labels again)
            
            #write thses new labels to a file
            with open(self.labels_file[:-4] + "_revised.csv",mode="w") as csvfile:
                csv_writer = csv.writer(csvfile,delimiter=',')
                for i in self.labels_list:
                    csv_writer.writerow([i])
            
            self.labels_list_counter = Counter(self.labels_list)

            adjacency_labels = self.labels_list[self.mesh.face_adjacency]

            self.adjacency_labels_col1, self.adjacency_labels_col2 = adjacency_labels.T
            #generate the vertices labels
            print(f"Time to generate new labels_list for soma caps: {time.time() - start_time}")
            start_time = time.time()
            self.generate_verts_to_face_dictionary(self.labels_list)
            print(f"Time to generate new dictionaries for soma caps: {time.time() - start_time}")
            
            start_time
            self.get_graph_structure()
            print("done creating connections -- %s seconds --" %(time.time() - start_time))
            

    def find_Apical(self,soma_index):
        """Returns the index of the most likely apical 
        1) calculate the height of 70% up the soma
        2) find all the neighbors of the soma using verts_to_Label
        3) filter out the neighbors that go below that
        4) filter away the neighbors that don't meet minimum number of face, height change and sdf median
        5) If multiple, pick the one that has the most number of neighbors


        """
        print("Soma Index = " + str(soma_index))
        print("Soma Connections = " + str(self.connections[soma_index]))
        mesh_Threshold = 2000
        height_Threshold =5000
        sdf_Threshold = 0.09
        #1) calculate the height of 70% up the soma (but have to adjust because the negative direction of y is 
        #direction of the apical), this new method gets the height of the first 30% of the somae which is actually
        # the top 30% of the soma once it is flipped in the right orientation
#         neighbor_verts = self.faces[np.where(self.labels_list == soma_index)]
#         soma_y_max = np.max(neighbor_verts,axis=0)[1]
#         soma_y_min = np.min(neighbor_verts,axis=0)[1]
        soma_verts = self.vertices[self.faces[np.where(self.labels_list == soma_index)].ravel()][:,1]
        soma_y_min = np.min(soma_verts)
        soma_y_max = np.max(soma_verts)
        self.soma_y_min = soma_y_min
        self.soma_y_max = soma_y_max
#         print("soma_y_max ="  + str(soma_y_max))
#         print("soma_y_min ="  + str(soma_y_min))
        
        
        soma_70_percent = (soma_y_max - soma_y_min)*0.3 +  soma_y_min
        print("soma_70_percent = " + str(soma_70_percent))
        #2) find all the neighbors of the soma using verts_to_Label
        soma_neighbors = self.connections[soma_index]
        #3) filter out the neighbors that go below that
        
        #find the maximum y value for the labels

        possible_Axons_filter_1 = [label for label in soma_neighbors 
                            if np.max(self.vertices[self.faces[np.where(self.labels_list == label)].ravel()][:,1]) < soma_70_percent]

        #4) filter away the neighbors that don't meet minimum number of face, height change and sdf median
        print("possible_Axons_filter_1 = " + str(possible_Axons_filter_1))
        possible_Axons_filter_2 = [lab for lab in possible_Axons_filter_1 if 
                                        self.mesh_Number[lab] > mesh_Threshold and 
        np.max(self.vertices[self.faces[np.where(self.labels_list == lab)].ravel()][:,1]) - np.min(self.vertices[self.faces[np.where(self.labels_list == lab)].ravel()][:,1]) > height_Threshold and
                                        self.sdf_final_dict[lab]["median"] > sdf_Threshold]
        print("possible_Axons_filter_2 = " + str(possible_Axons_filter_2))
        if len(possible_Axons_filter_2) <= 0:
            return "None"
        elif len(possible_Axons_filter_2) == 1:
            return possible_Axons_filter_2[0]
        else:
            #find the one with the most neighbors 
            ##### MIGHT WANT TO ADD IN WHERE FINDS THE THICKEST WIDTH !
            current_apical = possible_Axons_filter_2[0]
            current_apical_neighbors = len(self.connections[possible_Axons_filter_2[0]])
            for i in range(1,len(possible_Axons_filter_2)):
                if len(self.connections[possible_Axons_filter_2[i]]) > current_apical_neighbors:
                    current_apical = possible_Axons_filter_2[i]
                    current_apical_neighbors = len(self.connections[possible_Axons_filter_2[i]])

            return current_apical
    
    def classify_whole_neuron(self,possible_Apical,soma_index,threshold=700,axon_std_threshold=69):
        #check to see if no soma index
        if soma_index < 0:
            self.whole_neuron_labels ={lb:"unsure" for lb in self.connections.keys()}
            return
        
        #creates dictionary with unique labels whose value will store their final label
        whole_neuron_labels ={lb:"unsure" for lb in self.connections.keys()}
        whole_neuron_labels[soma_index] = "soma"

        #create a networkx graph based on connections
        G=nx.Graph(self.connections)

        
        #removes the soma from the list of nodes, but not actually remove it from the graph
        node_list = list(G.nodes)
        if(soma_index in node_list):
            node_list.remove(soma_index)
        else:
            #didn't find soma
            return []

        
        #finds the shortest path from any label to the soma
        shortest_paths = {}
        for node in node_list:
            shortest_paths[node] = [k for k in nx.shortest_path(G,node,soma_index)]

        #find the direct neighbors of the soma
        soma_branches = dict()
        
        soma_neighbors = self.connections[soma_index]
        
        
        print("soma_neighbors = " + str(soma_neighbors))
        
        #assemble each of these compartments into groups
        for node,path in shortest_paths.items():
            if possible_Apical not in path:
                specific_soma_neighbor = (set(path).intersection(set(soma_neighbors))).pop()
                
                if specific_soma_neighbor not in soma_branches.keys():
                    soma_branches[specific_soma_neighbor] = []
                soma_branches[specific_soma_neighbor].append(node)
        
        #return soma_branches
        
        print("soma_branches = " + str(soma_branches))
        #have groups of branches and assmble them into trimesh objects
        branches_submeshes = {}
        for group,group_list in soma_branches.items():
            total_indices = []
            for g in group_list:
                face_indices = np.where(self.labels_list == g)
                total_indices += face_indices
            
            #create a trimesh submshesh
            
            branches_submeshes[group] = self.mesh.submesh(total_indices,append=True)
        #return branches_submeshes
        
        #iterate through meshes and assign certain labels to these guys
        ## define certain thresholds for determining label
        cilia_threshold = 1000
        stub_threshold = 200
        non_dendrite_convex_threshold = 26.5
        
        
        #Calculate the soma 40%

        
        soma_height = self.soma_y_max - self.soma_y_min
        
        soma_lower_30 = self.soma_y_max - 0.3*soma_height
        print("self.soma_y_max = " + str(self.soma_y_max))
        print("self.soma_y_min = " + str(self.soma_y_min))
        print("soma_lower_30 = " + str(soma_lower_30))
        
        
        for neighbor,submesh in branches_submeshes.items():
            
            #get the number of faces
            total_faces = len(submesh.faces)
            #print(f"total_faces  = {total_faces}")
            
            if total_faces < stub_threshold:
                print(f"{neighbor} = stub soma")
                for x in soma_branches[neighbor]:
                    
                    whole_neuron_labels[x] = "soma"
            else:
            
                mean_convex = abs(np.mean(trimesh.convex.adjacency_projections(submesh)))
                #print(f"total_faces  = {mean_convex}")
                if mean_convex > non_dendrite_convex_threshold:
                    print("neighbor inside cilia check = " + str(neighbor))
                    #classify according to size

                    if total_faces < cilia_threshold:
                        print(f"{neighbor} = cilia")
                        for x in soma_branches[neighbor]:
                            whole_neuron_labels[x] = "cilia"
                    else:
                        print(f"{neighbor} = axon")
                        for x in soma_branches[neighbor]:
                            whole_neuron_labels[x] = "error"
                else: #try to see if there is any axon
                    #calculate the standard deviation
                    print("neighbor inside axon check = " + str(neighbor))
                    std_dev_convex = np.std((trimesh.convex.adjacency_projections(submesh)))
                    if neighbor == 83:
                        print("std_dev_convex = " + str(std_dev_convex))
                        print("axon_std_threshold = " + str(axon_std_threshold))
                    if std_dev_convex < axon_std_threshold:
                        #find the minimum y heght of neighbor
                        neighbor_y_min = np.min(self.vertices[self.faces[np.where(self.labels_list == neighbor)].ravel()][:,1])
                        neighbor_y_max = np.max(self.vertices[self.faces[np.where(self.labels_list == neighbor)].ravel()][:,1])
                        print("neighbor_y_min = " + str(neighbor_y_min))
                        print("neighbor_y_max = " + str(neighbor_y_max))
                        
                        if neighbor_y_min > soma_lower_30:
                            #make sure that it doesn't go higher than 40% soma height
                            for x in soma_branches[neighbor]:
                                whole_neuron_labels[x] = "axon"
                        else:
                            print(f"MET AXON THRESHOLD CRITERIA but not low enough on soma for neighbor = {neighbor}")
                            
        
        # checks if apical is present or not, and if not then just labels everything else basal
        if possible_Apical == "None":
            #label everything as basal if don't know
            for k,vals in whole_neuron_labels.items():
                if k != soma_index and vals == "unsure":
                    whole_neuron_labels[k] = "basal"
            self.whole_neuron_labels = whole_neuron_labels
            
            return 
                    
            
        #return branches_submeshes
        
        """ 4-29 added edition that will prevent small spines off of apical 
        from being considered oblique branches
        
    
        """
        #find the direct neighbors of the soma
        apical_branches = dict()
        
        apical_neighbors = self.connections[possible_Apical]
        apical_neighbors.remove(soma_index)
        
        
        #assemble each of these compartments into groups
        for node,path in shortest_paths.items():
            if possible_Apical in path and node != possible_Apical: #make sure only those obliques and not actual apical
                
                specific_apical_neighbor = (set(path).intersection(set(apical_neighbors))).pop()
                
                if specific_apical_neighbor not in apical_branches.keys():
                    apical_branches[specific_apical_neighbor] = []
                apical_branches[specific_apical_neighbor].append(node)
        
        #return soma_branches
        
        print("apical_branches = " + str(apical_branches))
        #have groups of branches and assmble them into trimesh objects
        branches_submeshes_apical = {}
        for group,group_list in apical_branches.items():
            total_indices = []
            for g in group_list:
                face_indices = np.where(self.labels_list == g)
                total_indices += face_indices
            
            #create a trimesh submshesh
            
            branches_submeshes_apical[group] = self.mesh.submesh(total_indices,append=True)
        #return branches_submeshes_apical
        
        #iterate through meshes and assign certain labels to these guys
        ## define certain thresholds for determining label
        
        stub_threshold_apical = 700
        apical_height_threshold = 0.75

        for neighbor,submesh in branches_submeshes_apical.items():
            
            #get the number of faces
            total_faces = len(submesh.faces)
            #print(f"total_faces  = {total_faces}")
            
            if total_faces < stub_threshold_apical:
                print(f"{neighbor} = stub apical")
                for x in apical_branches[neighbor]:
                    
                    whole_neuron_labels[x] = "apical"
            else:
            
                mean_convex = abs(np.mean(trimesh.convex.adjacency_projections(submesh)))
                #print(f"total_faces  = {mean_convex}")
                if mean_convex > non_dendrite_convex_threshold:
                    #classify according to size

                    print(f"{neighbor} = error")
                    for x in apical_branches[neighbor]:
                        whole_neuron_labels[x] = "error"
                else: #try to see if there is any axon
                    #calculate the standard deviation
                    std_dev_convex = np.std((trimesh.convex.adjacency_projections(submesh)))
                    if std_dev_convex < axon_std_threshold:
                        for x in apical_branches[neighbor]:
                            whole_neuron_labels[x] = "error"


        for label_name, path in shortest_paths.items():
            if label_name == possible_Apical: #labels the possible apical as apical
                whole_neuron_labels[label_name] = "apical"
            else:
                if possible_Apical in path:
                    #if has apical on path and not the apical itself, soma or other label --> label oblique
                    for jj in path: 
                        if jj != possible_Apical and jj != soma_index and whole_neuron_labels[jj] == "unsure":
                            whole_neuron_labels[jj] = "oblique" 
                else:
                    #if NO apical on path and not the apical itself, soma or other label --> label oblique
                    for jj in path:
                        if jj != possible_Apical and jj != soma_index and whole_neuron_labels[jj] == "unsure":
                            whole_neuron_labels[jj] = "basal" 

        #return the final list of labels:
        self.whole_neuron_labels = whole_neuron_labels
        return
    
    def label_whole_neuron(self):
        
        """
        iterates through all of faces and labels them accoring
        to the labels assigned to the cgal generic labels
        
        """
        
        #instead of going to datajoint for labels
        #this just have it locally so don't rely on datajoint

        apical_index = 2
        basal_index = 3
        oblique_index = 4
        soma_index = 5
        cilia_index = 12
        error_index = 10 
        axon_index=6


        self.final_faces_labels_list = np.zeros(len(self.faces))
        

        unknown_counter = 0

        for i,lab in enumerate(self.labels_list):
            #get the category according to the dictionary
            cat = self.whole_neuron_labels[lab]
            if cat == "apical":
                self.final_faces_labels_list[i] = apical_index
            elif cat == "basal":
                self.final_faces_labels_list[i] = basal_index
            elif cat == "oblique":
                self.final_faces_labels_list[i] = oblique_index
            elif cat == "soma":
                self.final_faces_labels_list[i] = soma_index
            elif cat == "cilia":
                self.final_faces_labels_list[i] = cilia_index
            elif cat == "axon":
                self.final_faces_labels_list[i] = axon_index
            elif cat == "error":
                self.final_faces_labels_list[i] = error_index
            else:
                #if wasn't labeled anything just assing it a random color based on cgal assignment
                self.output_faces_labels_list[i] = 18 + (int(lab))

        
    def generate_output_lists(self):
        """
        Will generate the final faces and vertices labels for the classification
        """
        #DON'T THINK I NEED THIS
#         face_Counter = Counter(self.final_faces_labels_list)
#         #print(face_Counter)

#         accepted_color_length = 18
#         random_labels = {int(l):int(accepted_color_length+i) for i,l in enumerate(face_Counter.keys()) if l >= accepted_color_length}
#         color_length = accepted_color_length
#         for i in range(0,color_length):
#             random_labels[i] = i


#         output_faces_list = [random_labels[int(ll)] for ll in final_faces_labels_list]
        output_faces_list = self.final_faces_labels_list
        

        #generate the vertices labels
        self.generate_verts_to_face_dictionary(output_faces_list)

        output_verts_list = [int(self.verts_to_Label[v][0]) for v in self.verts_to_Label]

        self.output_verts_labels_list = output_verts_list
        return self.final_faces_labels_list, self.output_verts_labels_list 
        
#     def label_cilia_axons_and_extract_branches(self,split_up_branches=True,soma_mesh=False,stub_threshold=0):
#         """
        
#         Process:
#         1) Divide the basal parts into seperate parts
#         2) Use the convex data and face side to decide what parts are errors, axons and cilia
#         a. All of these will have a higher convex number
#         b. The stubs will be the ones with face number below stub threshold
#         c. The axons will be those above axon threshold
#         d. Cillia will be those left over
        
#         Tricky part
#         For each part that needs to be relabeled, find the original face indices and relabel
        
#         Resplit the mesh after the relabeling
        
#         According to the parameters set return either the branches individually, as a whole or either with the soma seperately
        
        
#         """
        
#         pass


    def return_branches(self,stub_threshold=200,split_up_spines=True,shaft_mesh=False):
        apical_index = 2
        basal_index = 3
        oblique_index = 4
        soma_index = 5
        cilia_index = 12
        error_index = 10 
        axon_index=6
        
        basal_indexes = np.where(self.final_faces_labels_list == basal_index)[0]
        oblique_indexes = np.where(self.final_faces_labels_list == oblique_index)[0]
        apical_indexes = np.where(self.final_faces_labels_list == apical_index)[0]
        #axon_indexes = np.where(self.final_faces_labels_list == axon_index)
        spine_indexes = [np.concatenate([basal_indexes,oblique_indexes,apical_indexes])]
        
        spine_meshes_whole = self.mesh.submesh(spine_indexes,append=True)
        
        #decides if passing back spines as one whole mesh or seperate meshes
        if split_up_spines==True:
            individual_spines = []
            temp_spines = spine_meshes_whole.split(only_watertight=False)
            for spine in temp_spines:
                if len(spine.faces) >= stub_threshold:
                    individual_spines.append(spine)
        else:
            individual_spines = spine_meshes_whole
        #will also pass back the shaft of the mesh with the extracted spines
        if shaft_mesh==False:
            return individual_spines
        else:
            shaft_indexes = np.where(np.array(self.labels_list) == soma_index) 
            shaft_mesh_whole = self.mesh.submesh(shaft_indexes,append=True)
            return individual_spines,shaft_mesh_whole
        
        #divide into disconnected meshes and return this array
        return individual_spines
            

        
        
        
    
    
    
    
#     def extract_branches(self,labels_file_location,file_name,clusters,smoothness,
#                                            split_up_spines=True,shaft_mesh=False,**kwargs):


#             smooth_backbone_parameters = kwargs.pop('smooth_backbone_parameters', dict())
#             stub_threshold = kwargs.pop('stub_threshold', 50)


#             status = self.get_spine_classification(labels_file_location,file_name,clusters,
#                                           smoothness,smooth_backbone_parameters,stub_threshold)

#             if status != "Success":
#                 print("spine classification did not execute properly")
#                 return None

#             spine_indexes = np.where(np.array(self.labels_list) != -1)
#             spine_meshes_whole = self.mesh.submesh(spine_indexes,append=True)

#             #decides if passing back spines as one whole mesh or seperate meshes
#             if split_up_spines==True:
#                 individual_spines = []
#                 temp_spines = spine_meshes_wholes_whole.split(only_watertight=False)
#                 for spine in temp_spines:
#                     if len(spine.faces) >= stub_threshold:
#                         individual_spines.append(spine)
#             else:
#                 individual_spines = spine_meshes_whole

#             #will also pass back the shaft of the mesh with the extracted spines
#             if shaft_mesh==False:
#                 return individual_spines
#             else:
#                 shaft_indexes = np.where(np.array(self.labels_list) == -1) 
#                 shaft_mesh_whole = self.mesh.submesh(shaft_indexes,append=True)
#                 return individual_spines,shaft_mesh_whole
            
        
        
        
        
#         #divide into disconnected meshes and return this array
#         return individual_spines
            
    
    def clean_files(self):
        #clean the files 
        
        #1) new mesh file
        #2) cgal files (sdf and labels)
        pass
        
        
        

In [ ]:
#segment_id = 648518346349471500
#segment_id = 648518346349471156
global_start = time.time()
#classifier = WholeNeuronClassifier("./temp","neuron_" + str(segment_id) + ".off")
segment_id = 648518346349473045
segment_id = 648518346349386137
segment_id = 648518346349472574 #not correctly identifying the soma
#segment_id = 648518346349473045 #one correctly working for

classifier = WholeNeuronClassifier("./temp","neuron_" + str(segment_id) + ".off")
classifier.load_cgal_segmentation(clusters=4,smoothness=0.30)
#retrieves the cgal data
classifier.get_cgal_data_and_label_local_optomized()

#get the highest values of sdf
start_time = time.time()
classifier.get_highest_sdf_part(size_threshold=3000)

print("got highest part--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

#wants to generate the verts_to_face (have to do this again for the mesh)
classifier.generate_verts_to_face_dictionary()
print("done creating dictionaries -- %s seconds --" %(time.time() - start_time))

start_time = time.time()
#create a graph structure and stats for the whole neuron
classifier.get_graph_structure()
print("done creating connections -- %s seconds --" %(time.time() - start_time))

start_time = time.time()
soma_index=classifier.high_median
# neighbors_list = classifier.connections[soma_index].copy()

# neighbors_list.append(soma_index)
# # min_max = classifier.find_max_min_z_vals(neighbors_list,axis=1)
# # print("done FINDING NEIGHBOR STATS -- %s seconds --" %(time.time() - start_time))
# # #print("min_max = " + str(min_max))

start_time = time.time()
classifier.find_Soma_Caps(soma_index,min_width=0.23,max_faces=6000,max_n_connection=6,large_sized_convex=3)
print("done FINDING Soma Caps -- %s seconds --" %(time.time() - start_time))

start_time = time.time()
#send data to function that will find the Apical
possible_Apical = classifier.find_Apical(soma_index)
print("possible_Apical = " + str(possible_Apical))
print("done finding possible apicals -- %s seconds --" %(time.time() - start_time))

#use the apical label and the soma label to classify the rest as basal or oblique and return a dictionary that has the mapping of label to compartment type
#but only classifies the cgal labels and not each individual face
start_time = time.time()
branches = classifier.classify_whole_neuron(possible_Apical,soma_index,threshold=500,axon_std_threshold=69.5)
print("whole_neuron_labels = " + str(classifier.whole_neuron_labels))
print("giving neurons compartment label -- %s seconds --" %(time.time() - start_time))


#label the neurons according to classification
#############NEED TO ADD STEP THAT CALCULATES THE LABELS OF THE VERTICES ##################
start_time = time.time()
classifier.label_whole_neuron()
print("giving neuron compartment labels to faces -- %s seconds --" %(time.time() - start_time))



#####need to map the final_faces_labels_list to all successive numbers and get vertices
start_time = time.time()
output_faces_list, output_verts_list = classifier.generate_output_lists()
print("giving neuron compartment labels to vertices -- %s seconds --" %(time.time() - start_time))


dendritic_branches = classifier.return_branches()

print(f"Total time: {time.time() - global_start }")

#output the faces_labels to a file to test
# with open(str(segment_id) + "_classification.csv") as file:
    
#     csv_writer = csv
    
#             with open(labels_file) as csvfile:
#             #print("inside labels file")

#             for i,row in enumerate(csv.reader(csvfile)):
#                 triangles_labels[i] = int(row[0])










In [ ]:
classifier.min_max

In [ ]:
#classifier.labels_list
axon_indices = [11,14]
np.where(np.isin(classifier.labels_list,axon_indices))
classifier.labels_list[np.where(np.isin(classifier.labels_list,axon_indices))] = 6

In [ ]:
#output the faces_labels to a file to test
with open(str(segment_id) + "_classification.csv","w") as csvfile:
    
    csv_writer = csv.writer(csvfile)
    for i in output_faces_list:
        csv_writer.writerow([int(i)])


# NEED TO GET RID OF CAPS

In [ ]:
#checking the statistics of the caps:
caps_soma = [129,131]
caps_big_soma = [17]
caps_keep = [98]
regular_branches = [57,13,36]

In [ ]:
"""
Things that will help distinguish caps: 
1) No more than neighbors
2) Certain width
3) certain curvature

"""

#gather all of the caps that we want to get rid of 
caps_soma_mesh = {}
for i in caps_soma + caps_big_soma:
    new_mesh = classifier.mesh.submesh(np.where(classifier.labels_list == i))[0]
    caps_soma_mesh[i] = new_mesh

caps_dendrites_mesh = {}
for i in caps_keep + regular_branches:
    new_mesh = classifier.mesh.submesh(np.where(classifier.labels_list == i))[0]
    caps_dendrites_mesh[i] = new_mesh


In [ ]:
caps_soma_mesh[17].show()

In [ ]:
#compute statistics on soma group
print("Want to merge with soma")
for i,submesh in caps_soma_mesh.items():
    mean_convex = abs(np.mean(trimesh.convex.adjacency_projections(submesh)))
    n_faces = len(submesh.faces)
    width_data = classifier.sdf_final_dict[i]
    n_connections = len(classifier.connections[i])
    
    print(f"mean_convex = {mean_convex}, n_faces={n_faces}, width_data={width_data}, n_connections={n_connections}")
print("\n\n\n")
print("Those that don't want to merge with soma")
#compute statistics on soma group
for i,submesh in caps_dendrites_mesh.items():
    mean_convex = abs(np.mean(trimesh.convex.adjacency_projections(submesh)))
    n_faces = len(submesh.faces)
    width_data = classifier.sdf_final_dict[i]
    n_connections = len(classifier.connections[i])
    
    print(f" {i} mean_convex = {mean_convex}, n_faces={n_faces}, width_data={width_data}, n_connections={n_connections}")

In [ ]:
"""

things learned to help tell them apart:
1) maximum threshold for number of connections
2) Size threshold
3) Number of significant connections (size of the things attached)
4) Median width will be greater (0.240)

Final rules for identifying soma caps: 
1) median width > 23
2) n_faces < 7000
3) n_connections < 6


"""



# Checking the z value of the axon

In [ ]:
soma_index = 6

y_max = np.max(classifier.faces[np.where(classifier.labels_list == soma_index)],axis=0)[1]
print(y_max)
y_min = np.min(classifier.faces[np.where(classifier.labels_list == soma_index)],axis=0)[1]
print(y_min)

#classifier.faces

#mesh.submesh(np.where(classifier.labels_list == i))

# Checking the Axon

In [ ]:
axon_mesh = dendritic_branches[6]#std_dev_convex = np.std((trimesh.convex.adjacency_projections(axon_mesh)))
axon_mesh.show()

In [ ]:
axon_indices = [83]

In [ ]:
new_mesh = classifier.mesh.submesh(np.where(np.isin(classifier.labels_list,axon_indices)))[0]
new_mesh.show()

In [ ]:
#get the soma
soma_indices = [6]
soma_mesh = classifier.mesh.submesh(np.where(np.isin(classifier.labels_list,soma_indices)))[0]
soma_mesh.show()

In [ ]:
soma_min = np.min(soma_mesh.vertices[:,1])
soma_max = np.max(soma_mesh.vertices[:,1])
print(soma_min,soma_max)

In [ ]:
axon_min = np.min(axon_mesh.vertices[:,1])
axon_max = np.max(axon_mesh.vertices[:,1])
print(axon_min,axon_max)

In [ ]:
mean_convex = abs(np.std(trimesh.convex.adjacency_projections(new_mesh)))
if mean_convex < 69:
    print("hello")

In [ ]:
"""results from the test
self.soma_y_max = 222388.59375
self.soma_y_min = 202375.6875
soma_lower_30 = 216384.721875
neighbor_y_max = 204029.984375
"""

In [ ]:
#check that this object meets the standards of the axon:
std_dev_convex = np.std((trimesh.convex.adjacency_projections(new_mesh)))
print(std_dev_convex)

In [ ]:
#get the max and min of soma and axon mesh
axon_mesh

In [ ]:
branches[list(branches.keys())[10]].show()

In [ ]:
# loop that will do 

# CHECKING THE LABELING

# Testing if can distinguish axon branch from rest of the dendrites

In [ ]:
"""Notes on differentiation:
1) Can't tell the axon apart based on convex methods
2) Try sdf values



"""

In [ ]:
help(np.absolute)

In [ ]:
#Trying differentiation by curvature
for i,current_mesh in enumerate(dendritic_branches):
    print("branch " + str(i))
    mean_convex = np.mean(trimesh.convex.adjacency_projections(current_mesh))
    median_convex = np.median(trimesh.convex.adjacency_projections(current_mesh))
    #calculate the absolute values
    mean_convex_absolute = np.mean(np.absolute((trimesh.convex.adjacency_projections(current_mesh))))
    median_convex_absolute = np.median(np.absolute((trimesh.convex.adjacency_projections(current_mesh))))
    std_dev_convex = np.std((trimesh.convex.adjacency_projections(current_mesh)))
    std_dev_convex_absolute = np.std(np.absolute(trimesh.convex.adjacency_projections(current_mesh)))
    
    total_convex = sum(current_mesh.face_adjacency_convex)/len(current_mesh.face_adjacency_convex)
    print(f"mean_convex_absolute = {mean_convex_absolute}, median_convex_absolute = {median_convex_absolute}, total_convex = {total_convex}")
    print(f"mean_convex = {mean_convex}, median_convex = {median_convex}, total_convex = {total_convex}")
    print(f" std = {std_dev_convex}, Abs std = {std_dev_convex_absolute}")
    

In [ ]:
branches.keys()

In [ ]:
axon_submesh = branches[20]

In [ ]:
mean_convex = abs(np.mean(trimesh.convex.adjacency_projections(axon_submesh)))
mean_convex

# Testing the convex options for identifying axons from cilia

In [ ]:
#could be route for getting the indices of the mesh:
trimesh.graph.connected_components

In [ ]:
branches_list[4].face_adjacency_convex

In [ ]:
print(np.mean(np.absolute(trimesh.convex.adjacency_projections(branches_list[4]))))

In [ ]:
#get the projections of faces onto all of its neighboring faces
len(trimesh.convex.adjacency_projections(branches_list[4])),len(branches_list[4].face_adjacency),len(branches_list[4].face_adjacency_convex)

In [ ]:
little_faces = [11,12]
error_axons_indices = [3,13,14]
soma_index = [7]
cilia_index = [12]
dendrite_index = [i for i in range(len(branches_list)) if i not in (little_faces + error_axons_indices + soma_index + cilia_index) ]
dendrite_index

In [ ]:
#for each group gets some numbers on the convex adjacency
for i in error_axons_indices:
    current_mesh = branches_list[i]
    mean_convex = np.mean(trimesh.convex.adjacency_projections(current_mesh))
    median_convex = np.median(trimesh.convex.adjacency_projections(current_mesh))
    #calculate the absolute values
    mean_convex_absolute = np.mean(np.absolute((trimesh.convex.adjacency_projections(current_mesh))))
    median_convex_absolute = np.median(np.absolute((trimesh.convex.adjacency_projections(current_mesh))))
    
    total_convex = sum(current_mesh.face_adjacency_convex)/len(current_mesh.face_adjacency_convex)
    print(f"mean_convex_absolute = {mean_convex_absolute}, median_convex_absolute = {median_convex_absolute}, total_convex = {total_convex}")
    print(f"mean_convex = {mean_convex}, median_convex = {median_convex}, total_convex = {total_convex}")
    

In [ ]:
#for each group gets some numbers on the convex adjacency
for i in cilia_index:
    current_mesh = branches_list[i]
    mean_convex = np.mean(trimesh.convex.adjacency_projections(current_mesh))
    median_convex = np.median(trimesh.convex.adjacency_projections(current_mesh))
    #calculate the absolute values
    mean_convex_absolute = np.mean(np.absolute((trimesh.convex.adjacency_projections(current_mesh))))
    median_convex_absolute = np.median(np.absolute((trimesh.convex.adjacency_projections(current_mesh))))
    
    total_convex = sum(current_mesh.face_adjacency_convex)/len(current_mesh.face_adjacency_convex)
    print(f"mean_convex_absolute = {mean_convex_absolute}, median_convex_absolute = {median_convex_absolute}, total_convex = {total_convex}")
    print(f"mean_convex = {mean_convex}, median_convex = {median_convex}, total_convex = {total_convex}")
    

In [ ]:
#for each group gets some numbers on the convex adjacency
for i in dendrite_index:
    current_mesh = branches_list[i]
    mean_convex = np.mean(trimesh.convex.adjacency_projections(current_mesh))
    median_convex = np.median(trimesh.convex.adjacency_projections(current_mesh))
    #calculate the absolute values
    mean_convex_absolute = np.mean(np.absolute((trimesh.convex.adjacency_projections(current_mesh))))
    median_convex_absolute = np.median(np.absolute((trimesh.convex.adjacency_projections(current_mesh))))
    
    total_convex = sum(current_mesh.face_adjacency_convex)/len(current_mesh.face_adjacency_convex)
    print(f"mean_convex_absolute = {mean_convex_absolute}, median_convex_absolute = {median_convex_absolute}, total_convex = {total_convex}")
    print(f"mean_convex = {mean_convex}, median_convex = {median_convex}, total_convex = {total_convex}")
    

In [ ]:
Counter(classifier.final_faces_labels_list)

In [ ]:
output_faces_list

In [ ]:
faces_labels = classifier.labels_list
verts_labels = np.zeros(len(classifier.verts))

In [ ]:
# #write the neuron to the labels table:


# insert_key = dict(segment_id=segment_id,segmentation=2,decimation_ratio=0.35,
#                   clusters=classifier.clusters,
#                   smoothness=classifier.smoothness,date_time='2019-04-25 19:56:12',
#                  vertices = np.zeros(len(classifier.verts)),
#                  triangles = classifier.labels_list)


# ta3p100.AnnotationRevisited.insert1(insert_key,skip_duplicates=True)

In [ ]:
@schema
class AnnotationRevisited(dj.Manual):
    definition = """
    # creates the labels for the mesh table
    -> ta3p100.ComponentAutoSegmentWhole
    date_time  : timestamp   #going to keep constant
    ---
    vertices   : longblob     # label data for the vertices
    triangles  : longblob     # label data for the faces
    """